In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from numpy.polynomial import Polynomial
import statsmodels.formula.api as smf
%run Utilities_Indro.ipynb

excel_filename = 'F82H_Indro.xlsx'

# Load the data from the Excel file
uniform_elongation_col_names = ["T_CEA_u (K)", "CEA_u",\
                                "T_ECN_u (K)", "ECN_u",\
                                "T_CIEMAT_u (K)", "CIEMAT_u",\
                                "T_FZK_u (K)", "FZK_u"]

uniform_elongation_variables = ["T_CEA_u", "CEA_u",\
                                "T_ECN_u", "ECN_u",\
                                "T_CIEMAT_u", "CIEMAT_u",\
                                "T_FZK_u", "FZK_u"]

total_elongation_col_names = ["T_CEA_t (K)", "CEA_t",\
                              "T_ECN_t (K)", "ECN_t",\
                              "T_CIEMAT_t (K)", "CIEMAT_t",\
                              "T_FZK_t (K)", "FZK_t",\
                              "T_JAERI_t (K)", "JAERI_t"]

total_elongation_variables = ["T_CEA_t", "CEA_t",\
                              "T_ECN_t", "ECN_t",\
                              "T_CIEMAT_t", "CIEMAT_t",\
                              "T_FZK_t", "FZK_t",\
                              "T_JAERI_t", "JAERI_t"]

load_data(excel_filename, "Uniform Elongation", uniform_elongation_col_names, uniform_elongation_variables)
load_data(excel_filename, "Total Elongation", total_elongation_col_names, total_elongation_variables)

fsize = 14 # font size
msize = 10 # marker size

# Temperature range and initializations
T = np.arange(300, 1000, 10)
n = len(T)

# Concatenate and sort data for fitting
T_u, e_u = concatenate_and_sort([T_CEA_u, T_ECN_u, T_CIEMAT_u, T_FZK_u],\
                                [CEA_u, ECN_u, CIEMAT_u, FZK_u])

T_t, e_t = concatenate_and_sort([T_CEA_t, T_ECN_t, T_CIEMAT_t, T_FZK_t, T_JAERI_t],\
                                [CEA_t, ECN_t, CIEMAT_t, FZK_t, JAERI_t])

# Convert temperature to Kelvin
T_Ku = T_u + 273
T_Kt = T_t + 273

# Calculate confidence and prediction intervals
def get_conf_and_pred_intervals(x, y):
    fit_data = {'X': x, 'Y': y}
    fit_df = pd.DataFrame(fit_data)
    
    # Ordinary Least Squares
    model = smf.ols(formula='Y ~ I(X**3) + I(X**2) + X + 1', data=fit_df)
    results = model.fit()
    
    # Calculating prediction
    predictions = results.get_prediction(fit_df)
    prediction_summary_frame = predictions.summary_frame(alpha=0.05)
    
    return fit_df, results, prediction_summary_frame


# Plot Uniform Elongation of F82H
plt.figure('Uniform Elongation of F82H')
fit_df, results, prediction_summary_frame = get_conf_and_pred_intervals(T_Ku, e_u)
# Regression line
plt.plot(fit_df['X'], results.fittedvalues, color='black', label='Data Fit')
# Prediction interval
plt.fill_between(fit_df['X'], prediction_summary_frame['obs_ci_lower'], prediction_summary_frame['obs_ci_upper'], color='grey', alpha=0.2, label='95% Prediction Interval')
# Confidence interval
plt.fill_between(fit_df['X'], prediction_summary_frame['mean_ci_lower'], prediction_summary_frame['mean_ci_upper'], color='blue', alpha=0.2, label='95% Confidence Interval')
plt.xlabel('Temperature [K]', fontsize=14)
plt.ylabel(r'$\epsilon_\text{uniform}$ [%]', fontsize=fsize)
plt.xlim([290, 1000])
plt.ylim([0, 10])
plt.title('Uniform Elongation of F82H', fontsize=fsize)
plt.tick_params(axis='both', which='major', labelsize=14)    

plt.plot(T_CEA_u + 273, CEA_u, 'gs', markersize=msize, markerfacecolor='green', label='CEA')
plt.plot(T_ECN_u + 273, ECN_u, 'md', markersize=msize, markerfacecolor='magenta', label='ECN')
plt.plot(T_CIEMAT_u + 273, CIEMAT_u, 'ro', markersize=msize, markerfacecolor='red', label='CIEMAT')
plt.plot(T_FZK_u + 273, FZK_u, 'b>', markersize=msize, markerfacecolor='blue', label='FZK')
plt.legend(loc='upper left', ncol=2, fontsize=msize)

# Plot Total Elongation of F82H
plt.figure('Total Elongation of F82H')
fit_df, results, prediction_summary_frame = get_conf_and_pred_intervals(T_Kt, e_t)
# Regression line
plt.plot(fit_df['X'], results.fittedvalues, color='black', label='Data Fit')
# Prediction interval
plt.fill_between(fit_df['X'], prediction_summary_frame['obs_ci_lower'], prediction_summary_frame['obs_ci_upper'], color='grey', alpha=0.2, label='95% Prediction Interval')
# Confidence interval
plt.fill_between(fit_df['X'], prediction_summary_frame['mean_ci_lower'], prediction_summary_frame['mean_ci_upper'], color='blue', alpha=0.2, label='95% Confidence Interval')
plt.xlabel('Temperature [K]', fontsize=14)
plt.ylabel(r'$\epsilon_\text{total}$ [%]', fontsize=14)
plt.xlim([290, 1000])
plt.ylim([0, 50])
plt.title('Total Elongation of F82H', fontsize=14)
plt.tick_params(axis='both', which='major', labelsize=14)    

plt.plot(T_CEA_t + 273, CEA_t, 'gs', markersize=msize, markerfacecolor='green', label='CEA')
plt.plot(T_ECN_t + 273, ECN_t, 'ro', markersize=msize, markerfacecolor='red', label='ECN')
plt.plot(T_CIEMAT_t + 273, CIEMAT_t, 'md', markersize=msize, markerfacecolor='magenta', label='CIEMAT')
plt.plot(T_FZK_t + 273, FZK_t, 'b>', markersize=msize, markerfacecolor='blue', label='FZK')
plt.plot(T_JAERI_t + 273, JAERI_t, 'g<', markersize=msize, markerfacecolor='green', label='JAERI')
plt.legend(loc='upper left', ncol=2, fontsize=10)

ValueError: 
\epsilon_\text{uniform}
           ^
Expected {accentprefixed | accent | symbol | c_over_c | function | group | frac | dfrac | binom | genfrac | overset | underset | sqrt | overline | operatorname}, found 'ext'  (at char 11), (line:1, col:12)

<Figure size 432x288 with 1 Axes>

ValueError: 
\epsilon_\text{total}
           ^
Expected {accentprefixed | accent | symbol | c_over_c | function | group | frac | dfrac | binom | genfrac | overset | underset | sqrt | overline | operatorname}, found 'ext'  (at char 11), (line:1, col:12)

<Figure size 432x288 with 1 Axes>